In [15]:
import pandas as pd

In [16]:
import gzip

# Load the Wikipedia article titles into a set for efficient lookup
wiki_titles = set()
with gzip.open("enwiki-latest-all-titles-in-ns0.gz", "rt", encoding="utf-8") as f:
    for title in f:
        title = title.strip()
        title = title.replace("_", " ")
        wiki_titles.add(title.strip())

# Check if a person name has a Wikipedia article associated with it
person_name = "Bill Clinton"
if person_name in wiki_titles:
    print(f"{person_name} has a Wikipedia article!")
else:
    print(f"{person_name} does not have a Wikipedia article.")

Bill Clinton has a Wikipedia article!


In [17]:
import wikipedia

# set the language for the Wikipedia API
wikipedia.set_lang("en")

In [19]:
filename = 'data_2014'
df = pd.read_csv(f'{filename}_with_topics_full.csv')
# df = pd.read_json(filename)

for index, row in df.iterrows():
    title = row['title']
    print(title)

    # ask for user input
    while True:
        usertext = input("Enter a person's name: ")
        # check if the title is in the set
        if usertext in wiki_titles:
            print(f"{usertext} has a Wikipedia article!")
            break
        elif usertext == 'q':
            break
        else:
            titles = wikipedia.search(usertext)
            print(f"{usertext} does not have a Wikipedia article. Did you mean: {titles}?")

    df.at[index, 'wiki'] = usertext


I am Wolfgang Gartner. Ask me anything you want.
Wolfgang Gartner has a Wikipedia article!
We are the Pornhub team. Ask Us Anything.
Pornhub has a Wikipedia article!
I am Hans Zimmer, back on reddit once more. Ask me anything!
sssss does not have a Wikipedia article. Did you mean: ['Sssssss', 'Navon figure', '2gether (Thai TV series)', 'Hemachandra', 'Production code number', 'Shotgun shell', 'Scrubb (band)', 'Moonwalk (dance)', 'Typographical Number Theory', 'Hexasulfur']?
Hans Zimmer has a Wikipedia article!
I'm Greg Bristol, retired FBI Special Agent fighting human trafficking. AMA!
FBI has a Wikipedia article!
This is Fat Mike from NOFX. I got tricked into doing this, shit! AMA
NOFX has a Wikipedia article!
I am Bryan Caplan, economist and professor at George Mason University and advocate of free immigration and free trade. AMA!
sssss does not have a Wikipedia article. Did you mean: ['Sssssss', 'Navon figure', '2gether (Thai TV series)', 'Hemachandra', 'Production code number', 'Sh

In [21]:
df[['title', 'wiki']].to_csv(f'{filename}_wiki.csv')

In [22]:
df_wiki = pd.read_csv(f'{filename}_wiki.csv')

In [23]:
import numpy as np

In [24]:
import wikipediaapi

# Initialize the Wikipedia API
wiki_api = wikipediaapi.Wikipedia('en')

In [25]:
# grab the first paragraph from the summary of the Wikipedia article
for index, row in df_wiki.iterrows():
    title = row['title']
    wiki = row['wiki']
    if wiki in ['Request', 'Story']:
        df_wiki.at[index, 'summary'] = np.nan
        continue
    try:
        summary = wiki_api.page(wiki).summary.split('\n')[0]
        df_wiki.at[index, 'summary'] = summary
        # print(f"{wiki} summary: {summary}")
    except:
        print(f"Error: {wiki} does not have a summary.")

In [26]:
df_wiki.to_csv(f'{filename}_wiki_summary.csv')

In [27]:
df_wiki = pd.read_csv(f'{filename}_wiki_summary.csv')

In [28]:
import re
import nltk

nltk.download('punkt') 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ehsanulkabir/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
# replace nan summaries with empty strings
df_wiki['summary'] = df_wiki['summary'].fillna('')

# only keep as many sentences as needed to have a summary that is more than 50 tokens
def get_first_sentence(summary):
    sentences = nltk.sent_tokenize(summary)
    for i, sentence in enumerate(sentences):
        if len(nltk.word_tokenize(' '.join(sentences[:i]))) > 50:
            return ' '.join(sentences[:i])
    return summary

df_wiki['summary'] = df_wiki['summary'].apply(get_first_sentence)

# remove non-engish characters
df_wiki['summary'] = df_wiki['summary'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', x))

# collapse multiple spaces into one
df_wiki['summary'] = df_wiki['summary'].apply(lambda x: re.sub(r'\s+', ' ', x))


In [30]:
df_wiki.to_csv(f'{filename}_wiki_summary_clean.csv')

In [31]:
df_wiki = pd.read_csv(f'{filename}_wiki_summary_clean.csv')

In [32]:
# preprocess title
# replace 'iama' in any variation of capital/lowercase letters with 'I am a'
df_wiki['title'] = df_wiki['title'].apply(lambda x: re.sub(r'iama ', 'I am a ', x, flags=re.IGNORECASE))

# replace ' ama' in any variation of capital/lowercase letters with ''
df_wiki['title'] = df_wiki['title'].apply(lambda x: re.sub(r' ama', '', x, flags=re.IGNORECASE))

# replace ' aua' in any variation of capital/lowercase letters with ''
df_wiki['title'] = df_wiki['title'].apply(lambda x: re.sub(r' aua', '', x, flags=re.IGNORECASE))

# replace '&amp;' with '&'
df_wiki['title'] = df_wiki['title'].apply(lambda x: re.sub(r'&amp;', '&', x))

In [33]:
# find a list of non-alphanumeric characters in the title
non_alphanumeric = df_wiki['title'].apply(lambda x: re.findall(r'[^a-zA-Z0-9 ]', x))
# merge all the lists into one list
non_alphanumeric = [item for sublist in non_alphanumeric for item in sublist]
# remove duplicates
non_alphanumeric = list(set(non_alphanumeric))

safe_characters = [' ', '&', '-', "'", '.', ',', '!', '?', ':', ';', '(', ')', '[', ']', '/', '"', '%', '$', '@', '#', '+', '=']

# remove safe characters from the list of non-alphanumeric characters
for char in safe_characters:
    if char in non_alphanumeric:
        non_alphanumeric.remove(char)

# remove non-alphanumeric characters from the title
def remove_non_alphanumeric(title):
    for char in non_alphanumeric:
        title = title.replace(char, '')
    return title

df_wiki['title'] = df_wiki['title'].apply(remove_non_alphanumeric)

In [34]:
# function to truncate after 'Ask' in the title
def truncate_title(title):
    # find the index of 'Ask' in the title
    index = title.lower().find('ask ')
    # if 'Ask' is not found, return the original title
    if index == -1:
        return title
    # otherwise, return the title truncated after 'Ask'
    else:
        return title[:index]
    
df_wiki['title'] = df_wiki['title'].apply(truncate_title)

# remove leading and trailing whitespace
df_wiki['title'] = df_wiki['title'].apply(lambda x: x.strip())

# replace consecutive punctuation with the last punctuation
df_wiki['title'] = df_wiki['title'].apply(lambda x: re.sub(r'[.,?!:;]+(?=[.,?!:;])', '', x))

In [35]:
df_wiki.to_csv(f'{filename}_wiki_summary_title_clean.csv')

In [36]:
df_wiki = pd.read_csv(f'{filename}_wiki_summary_title_clean.csv')

In [39]:
# replace nan summary and title with empty strings
df_wiki['summary'] = df_wiki['summary'].fillna('')
df_wiki['title'] = df_wiki['title'].fillna('')

In [40]:
# combine the title and summary into one column: context
df_wiki['context'] = df_wiki.apply(lambda x: x['title'] + ' ' + x['summary'], axis=1)

In [41]:
# make context empty string if there is no summary
df_wiki['context'] = df_wiki.apply(lambda x: '' if x['summary'] == '' else x['context'], axis=1)

In [42]:
df_wiki[['context', 'title']].to_csv(f'{filename}_with_context.csv')

In [43]:
df_wiki['context'].to_numpy()

array(['I am Wolfgang Gartner. Joseph Thomas Youngman (born March 17, 1982), better known by his stage name Wolfgang Gartner, is an American DJ and music producer. Much of his music prior to 2010 was released through his own record label, Kindergarten, but he signed with Ultra Records in 2010 and Ministry of Sound in the UK in 2011.',
       'We are the Pornhub team. Pornhub is a Canadian-owned internet pornography website. It is one of several pornographic video-streaming websites owned by MindGeek. As of November 2022, Pornhub is the 13th-most-trafficked website in the world and the second-most-trafficked adult website after XVideos.Pornhub was launched in Montreal, Canada, in 2007. Pornhub also has an office and servers in Limassol, Cyprus.',
       'I am Hans Zimmer, back on reddit once more. Hans Florian Zimmer (German pronunciation: [hans floian tsm] (listen); born 12 September 1957) is a German film score composer and music producer. He has won two Oscars and four Grammys, and h